# Grouping, resampling and aggregations

The hourly data over 23 years contain about ~800,000 observations. It is not viable to extract any knowledge by just looking at them ony by one. Thus an analysis of this sort of data almost always consists of proper **grouping** (selecting which rows we want to condense) and **aggregation** (applying an operation to produce a single or a handful of values from them).

In [ ]:
# Necessary import evil

import pandas as pd
from pandas import IndexSlice as idx, Grouper
import pandera as pa
from pandera.typing import Int16, DataFrame, Series, Category

from weatherlyser.loader import load_chmi_data

In [ ]:
# Load the saved data
data = pd.read_parquet("./data/open_meteo_2000-2022.pq")

data.head()

In [ ]:
# We will skip this now as it takes a lot of time!
# HistoricalWeatherDataFrame.validate(data)

In [ ]:
# Most meteorological reporting is done in local time
data = data.tz_convert(level="time", tz="Europe/Prague")

For simplicity, we will be working with the "best model" for now. 

In [ ]:
best_model_data = data.loc[idx[:, "best_match"], :].reset_index("model", drop=True)
best_model_data.head()

The weather description will become handy for our first aggregations.

In [ ]:
from weatherlyser.pa_models import WEATHER_CODES


class WeatherCodeModel(pa.DataFrameModel):
    weathercode: Int16 = pa.Field(ge=0, le=99, nullable=True)

@pa.check_types
# *** Annotations as a task
def get_weather_description(df: DataFrame[WeatherCodeModel]) -> Series[Category]:
    """Converts the weather code column in a Pandas DataFrame
    to a categorical column with the weather descriptions as the category levels.

    Args:
        df: A Pandas DataFrame with a column called "weathercode".

    Returns:
        A categorical series with a string representation of the code.
    """
    # *** The mapping as a task
    return df["weathercode"].map(WEATHER_CODES).astype("category")

get_weather_description(best_model_data)

In [ ]:
best_model_data = best_model_data.assign(
    description=get_weather_description(best_model_data)
)

## Grouping and simple aggregations

In [ ]:
grouped = best_model_data.groupby("description")
grouped

What is this object? Is this some numerical result yet? No, it is actually just the segmentation of the dataframe into **groups** of **rows** that will be handled by subsequent operation.

The groupby mechanism ensures that the operations done on the sub-dataframes are recombined together with the grouping key(s) as index (levels of index):

In [ ]:
# Select a column and do a mean over it, within the categories
grouped["rain"].mean()

In [ ]:
# Multiple aggregations over the same column
grouped["rain"].agg(["min", "mean", "max"])

## Resampling

With time series, we usually do not want to group over (all) values of column, but over some interval (typically of the index or index level). The mechanism is pretty much the same but the method is called `resample` and we need an extra argument, frequency.

So in order to get each year's total precipitation:

In [ ]:
best_model_data.resample("1Y")  # 1Y = every one year

This, in parallel to the groupby result is an object just holding the segmentations and any aggregation must be done on it:

In [ ]:
best_model_data.resample("1Y")["precipitation"].sum()

We will not be plotting much in this workshop but the simple `plot` method will still come handy to see what we calculated.

In [ ]:
best_model_data.resample("1Y")["precipitation"].sum().plot()

**Exercise:** Write a function `get_daily_temperature_stats` that finds the minimum, mean and maximum temperatures for each day. (Bonus: You can annotate it with a pandera model).

In [ ]:
def get_daily_temperature_stats(df):
   ...

get_daily_temperature_stats(best_model_data)

Sometimes, we want to combine grouping and resampling in one operation (although a nested grouping will probably work too). The **`Grouper`** class, used instead of strings as the `groupby` argument, allows to combine columns and normal/time index levels: 

In [ ]:
data.groupby([Grouper(level="time", freq="1D"), Grouper(level="model")])["precipitation"].sum() #.plot()

In [ ]:
# Have names for the aggregations: name=>(field, agg.method)
data.groupby([Grouper(level="time", freq="1D"), Grouper(level="model")]).agg(
    min_pressure=("surface_pressure", "min")
)

**Exercise:** Can you modify your get_daily_temperature_stats so that it works for the entire data, including model?

In [ ]:
def get_daily_temperature_stats2(df):



    min_temp = ("temp", "min")
    ...

get_daily_temperature_stats2(data)

### Feature engineering



When looking at periodically repeating phenomena, we quite often do not want to "just" resample but look at aggregations over all the repeating intervals, such as when we are interested in (historical) mean temperature in May. The `resample` or `groupby` (even with Grouper) will not work straightaway.

The easiest way is to construct helper columns with the time series feature (such as month number).

In [ ]:
data.index.get_level_values(0)

In [ ]:
mean_temperature_per_month = best_model_data.assign(
    month=lambda df: df.index.get_level_values(0).month
).groupby("month")["temperature_2m"].mean()

mean_temperature_per_month

In [ ]:
mean_temperature_per_month.plot(kind="bar")

**Exercise** For each calendar month (i.e. January, ...), find the typical (mean) hourly evolution of temperature over the course of the day. For that, you can use the attributes of the index and the method `unstack` to turn a level of a hierarchical index into columns (for each month).

In [ ]:
def daily_monthly_pattern(df, column="temperature"):
    result = ...
    return result.unstack("month")

In [ ]:
daily_monthly_pattern(best_model_data).plot()
daily_monthly_pattern(best_model_data)

**Complex exercise**: For each year, find how many minimum and maximum temperature days it has (the media tell us the weather is becoming more and more extreme!)

We will do this in multiple steps:

* Construct features "year", "month" and "day".
* Group over those and find the extremes, use `idxmax`, `idxmin` methods.
* Extract the year from both series (beware, that you need to use the `.dt` accessor for series datetime methods)
* Use value_counts or groupby to count the year occurences

Optional parts:
* Create a DataFrameModel checking the output: int as index (should be consecutive years somewhere around 2000), two columns (min_count, max_count?) as positive integes
* Create a hypothesis test showing that the total number of extremes does not exceed 366 over a dataframe with more than one-year worth of data

In [ ]:
def find_year_extremes(temperature_data: pd.DataFrame) -> pd.DataFrame:
    # Create the year, month, day features
    temperature_With_features = ...
    
    # For each month / day, get the extreme values
    extreme_dates = ...

    # Extract the years component
    extreme_years = ...

    # Do the final counting
    extreme_counts = ...

    return extreme_counts


find_year_extremes(temperature_data=get_daily_temperature_stats(best_model_data))

In [ ]:
# On CHMI data

chmi_data = load_chmi_data()
chmi_data_renamed = chmi_data.rename(columns={"minimum_temperature": "min", "maximum_temperature": "max"})
find_year_extremes(chmi_data_renamed)